In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
path_df = "/home/sushant/Downloads/saarthi/task_data/train_data.csv"
with open(path_df, 'rb') as data:
    df = pd.read_csv(data)

In [3]:
df.head()

,path,transcription,action,object,location
0,wavs/speakers/xRQE5VD7rRHVdyvM/7372ca00-45c4-1...,Turn on the kitchen lights,activate,lights,kitchen
1,wavs/speakers/R3mexpM2YAtdPbL7/dae28110-44fe-1...,Turn up the temperature,increase,heat,none
2,wavs/speakers/ZebMRl5Z7dhrPKRD/b55dcfd0-455d-1...,OK now switch the main language to Chinese,change language,Chinese,none
3,wavs/speakers/ppzZqYxGkESMdA5Az/61c54a20-4476-...,Turn down the bathroom temperature,decrease,heat,washroom
4,wavs/speakers/zaEBPeMY4NUbDnZy/8ef57ec0-44df-1...,Change the language,change language,none,none


In [4]:
df.loc[0]['transcription']

'Turn on the kitchen lights'

In [5]:
# Making everything to lower case
df['Content_Parsed_1'] = df['transcription'].str.lower()

In [6]:
df.head()

,path,transcription,action,object,location,Content_Parsed_1
0,wavs/speakers/xRQE5VD7rRHVdyvM/7372ca00-45c4-1...,Turn on the kitchen lights,activate,lights,kitchen,turn on the kitchen lights
1,wavs/speakers/R3mexpM2YAtdPbL7/dae28110-44fe-1...,Turn up the temperature,increase,heat,none,turn up the temperature
2,wavs/speakers/ZebMRl5Z7dhrPKRD/b55dcfd0-455d-1...,OK now switch the main language to Chinese,change language,Chinese,none,ok now switch the main language to chinese
3,wavs/speakers/ppzZqYxGkESMdA5Az/61c54a20-4476-...,Turn down the bathroom temperature,decrease,heat,washroom,turn down the bathroom temperature
4,wavs/speakers/zaEBPeMY4NUbDnZy/8ef57ec0-44df-1...,Change the language,change language,none,none,change the language


In [7]:
# removing punctuation signs
punctuation_signs = list("?:!.,;")
df['Content_Parsed_2'] = df['Content_Parsed_1']

for punct_sign in punctuation_signs:
    df['Content_Parsed_2'] = df['Content_Parsed_2'].str.replace(punct_sign, '')

<ipython-input-7-9ea803286063>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Content_Parsed_2'] = df['Content_Parsed_2'].str.replace(punct_sign, '')


In [8]:
# removing possessives
df['Content_Parsed_3'] = df['Content_Parsed_2'].str.replace("\'s", "")


In [9]:
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /home/sushant/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
wordnet_lemmatizer = WordNetLemmatizer()

In [11]:
# Lemmatizing the content
nrows = len(df)
lemmatized_text_list = []
for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_3']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)


In [12]:
df['Content_Parsed_4'] = lemmatized_text_list

In [13]:
df.head(15)

,path,transcription,action,object,location,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4
0,wavs/speakers/xRQE5VD7rRHVdyvM/7372ca00-45c4-1...,Turn on the kitchen lights,activate,lights,kitchen,turn on the kitchen lights,turn on the kitchen lights,turn on the kitchen lights,turn on the kitchen light
1,wavs/speakers/R3mexpM2YAtdPbL7/dae28110-44fe-1...,Turn up the temperature,increase,heat,none,turn up the temperature,turn up the temperature,turn up the temperature,turn up the temperature
2,wavs/speakers/ZebMRl5Z7dhrPKRD/b55dcfd0-455d-1...,OK now switch the main language to Chinese,change language,Chinese,none,ok now switch the main language to chinese,ok now switch the main language to chinese,ok now switch the main language to chinese,ok now switch the main language to chinese
3,wavs/speakers/ppzZqYxGkESMdA5Az/61c54a20-4476-...,Turn down the bathroom temperature,decrease,heat,washroom,turn down the bathroom temperature,turn down the bathroom temperature,turn down the bathroom temperature,turn down the bathroom temperature
4,wavs/speakers/zaEBPeMY4NUbDnZy/8ef57ec0-44df-1...,Change the language,change language,none,none,change the language,change the language,change the language,change the language
5,wavs/speakers/W4XOzzNEbrtZz4dW/8a1fedc0-44d2-1...,Turn sound down,decrease,volume,none,turn sound down,turn sound down,turn sound down,turn sound down
6,wavs/speakers/8e5qRjN7dGuovkRY/92366b20-459e-1...,Decrease the heating in the washroom,decrease,heat,washroom,decrease the heating in the washroom,decrease the heating in the washroom,decrease the heating in the washroom,decrease the heat in the washroom
7,wavs/speakers/W4XOzzNEbrtZz4dW/d40180c0-44d2-1...,"It’s too loud, turn it down",decrease,volume,none,"it’s too loud, turn it down",it’s too loud turn it down,it’s too loud turn it down,it’s too loud turn it down
8,wavs/speakers/g2dnA9Wpvzi2WAmZ/694d7090-454a-1...,Change language,change language,none,none,change language,change language,change language,change language
9,wavs/speakers/5BEzPgPKe8taG9OB/b9b56e80-457b-1...,Turn the lights on in the kitchen,activate,lights,kitchen,turn the lights on in the kitchen,turn the lights on in the kitchen,turn the lights on in the kitchen,turn the light on in the kitchen


In [14]:
df.to_csv('./modified_train.csv')

In [15]:
glove_path ="/home/sushant/Downloads/saarthi"

In [16]:
import bcolz

In [17]:
vectors = bcolz.open(f'{glove_path}/6B.50.dat')[:]
words = pickle.load(open(f'{glove_path}/6B.50_words.pkl', 'rb'))
word2idx = pickle.load(open(f'{glove_path}/6B.50_idx.pkl', 'rb'))
glove = {w: vectors[word2idx[w]] for w in words}

In [18]:
target_vocab = {}
nrows = len(df)
for row in range(0, nrows):
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")
    for word in text_words:
        target_vocab[str(word)] = 1

In [19]:
nrows = len(df)
for row in range(0, nrows):
    text = df.loc[row]['action']
    text_words = text.split(" ")
    for word in text_words:
        target_vocab[str(word)] = 1

In [20]:
nrows = len(df)
for row in range(0, nrows):
    text = df.loc[row]['object']
    text_words = text.split(" ")
    for word in text_words:
        target_vocab[str(word)] = 1

In [21]:
nrows = len(df)
for row in range(0, nrows):
    text = df.loc[row]['location']
    text_words = text.split(" ")
    for word in text_words:
        target_vocab[str(word)] = 1

In [22]:
print(target_vocab.keys())

dict_keys(['turn', 'on', 'the', 'kitchen', 'light', 'up', 'temperature', 'ok', 'now', 'switch', 'main', 'language', 'to', 'chinese', 'down', 'bathroom', 'change', 'sound', 'decrease', 'heat', 'in', 'washroom', 'it’s', 'too', 'loud', 'it', 'bedroom', 'increase', 'set', 'i', 'need', 'practice', 'my', 'english', 'hear', 'this', 'volume', 'lamp', 'off', 'bring', 'me', 'shoe', 'get', 'newspaper', "can't", 'that', 'fetch', 'sock', 'system', 'make', 'music', 'louder', 'phone', 'go', 'lower', 'could', 'you', 'korean', 'far', 'max', 'some', 'juice', 'less', 'please', 'device', 'put', 'video', 'be', 'low', 'play', 'audio', 'quieter', 'languages', 'quiet', 'resume', 'use', 'a', 'different', 'mute', 'hotter', 'pause', 'level', 'german', 'start', 'settings', 'stop', 'softer', 'that’s', 'more', 'reduce', 'allow', "couldn't", 'anything', 'cooler', 'open', 'activate', 'deactivate', 'lights', 'Chinese', 'none', 'English', 'shoes', 'socks', 'Korean', 'German'])


In [23]:
matrix_len = len(target_vocab.keys())
weights_matrix = np.zeros((matrix_len, 50))
words_found = 0

for i, word in enumerate(target_vocab.keys()):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(50, ))

In [24]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [25]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.lstm_output = nn.LSTM(hidden_dim, hidden_dim)


    def forward(self, sentence):
        lstm_out, _ = self.lstm(sentence)
        lstm_out = lstm_out[-1, :, :]
        out = []
        for i in range(3):
            o = self.lstm_output(lstm_out)
            out.append(o)
            lstm_out = o
        return torch.tensor(out).unsqueeze(0)
        
        

In [26]:
def sentence_to_tensor(sentence):
    l = sentence.split(" ")
    t = []
    for word in l:
        t.append(glove[word])
    return torch.tensor(t).unsqueeze(0).double()

In [27]:
sentence = df.loc[0]['Content_Parsed_4']

In [28]:
t = sentence_to_tensor(sentence)
print(t.shape)

torch.Size([1, 5, 50])


In [29]:
model = LSTMTagger(50, 20)
model(t)

RuntimeError: expected scalar type Double but found Float